# Scrape HTML Entity Page from Wikipedia

In [ ]:
from bs4 import BeautifulSoup
import requests, re
import pandas as pd

In [ ]:
URL = 'https://en.wikipedia.org/wiki/List_of_XML_and_HTML_character_entity_references'

WikiPedia is fussy about bots so we need to spoof headers.

In [ ]:
headers = {
    'User-Agent': 'Lynx/2.8.6rel.5 libwww-FM/2.14'
}

In [ ]:
with requests.get(URL, headers=headers) as r:
    soup = BeautifulSoup(r.text, 'html.parser')

There are two tables on the page. We want the second one.

In [ ]:
html = soup.find_all("table", attrs = {'class': 'wikitable'})[1]

We could probably have used the `.jquery-tablesorter` selector as well.

Find the table dimensions.

In [ ]:
rows = html.find_all("tr")
rows

In [ ]:
# Number of rows (subtracting one for headers).
#
nrows = len(rows) - 1

In [ ]:
# Number of columns (from longest row).
#
row_lengths = [len(tr.find_all(['th', 'td'])) for tr in rows]
ncols = max(row_lengths)

In [ ]:
'Table has %d rows and %d columns.' % (nrows, ncols)

In general tables can be rather complicated, with merged cells. You might need to look out for `colspan` and `rowspan` attributes. This table is nice and simple though.

## Write to CSV

In [ ]:
col_index = [0, 1, 2, 6]

In [ ]:
with open('wikipedia-html-entity.csv', 'w') as f:
    for tr in rows:
        # Find all header or data cells.
        cells = tr.find_all(['th', 'td'])
        # Select specific columns.
        cells = [cells[i] for i in col_index]
        f.write('|'.join([cell.text for cell in cells])+'\n')

**Note:** This CSV file will require some manual work because it has unmatched quote characters which play havoc with spreadsheets.

## Creating a Data Frame

Find the column headers. We'll obviously need to clean these up.

In [ ]:
rows[0].find_all("th")

Prepare the headers by converting to lowercase, removing footnotes and retaining only first word.

In [ ]:
def prepare_header(th):
    th = th.text
    th = th.lower()
    th = re.sub('\[[^\]]*\]', '', th)
    th = th.split()[0]
    return(th)

column_names = [prepare_header(th) for th in rows[0].find_all("th")]

### Populating from a List

Populate a two dimensional list from the table cells.

In [ ]:
table_data = [[td.text for td in tr.find_all('td')] for tr in rows[1:]]

Then create the data frame directly from the list.

In [ ]:
df = pd.DataFrame(table_data, columns=table_names)

Again we select only a subset of columns.

In [ ]:
df = df[['name', 'character', 'unicode', 'description']]
df.head()

### Quick Manipulations on Data Frame

There's a method to write the entire data frame to a CSV file. This is a much more elegant approach to writing the CSV and immediately produces a valid file.

In [ ]:
df.to_csv('wikipedia-html-entity.csv', sep='|')

Extract the column names.

In [ ]:
df.columns

In [ ]:
df.dtypes

Check for missing values.

In [ ]:
df.isnull().sum()

Slice out a column.

In [ ]:
df['name']